>I don't use folksongs, folksongs use me
>>Velljo Tormis, *regilaul* revitalist composer

# Stress in Estonian
# Kalevala Meter 
# Research Questions
# Previous Work
# Hypotheses


# Methods

This section is outlined as follows: 
1. the *regilaul* corpus is described. 
    1. singers/informants background info
    2. physical recording info (collection/collectors)
2. Then, the annotation process is described. 
    1. beat detective: 
        1.Logic method 
        2. Nyquist and Audacity 
        3. (manual check: definition of a beat)
    2. midi to textgrid (annotate silences script) 
    3. txt file of lyrics to phrases with praatio (check: definition of a phrase, verses instead of choruses decision?)
    4. forced aligner: word level (check: definition of word)
    5. syllable level (coded for stress and quanitity!) with varbomorf manual check: definition of a syllable (use ictus/off-ictus, duh!)
    6. segmental level (run from syllable level) (manual check: definition of each segment and the segment's environmental criteria to be included in analysis)
3. The measurement process is described 
    1. temporal aspects: 
        1. vowel duration
        2. consonant duration 
        3. syllable duration 
    2. spectral aspects: 
        1. vowel space
        2. spectral tilt





1. the *regilaul* corpus is described. 
    1. singers/informants background info
    2. physical recording info (collection/collectors)
2. Then, the annotation process is described. 
    1. beat detective: 
        1.Logic method 
        2. Nyquist and Audacity 
        3. (manual check: definition of a beat)
 2. midi to textgrid (annotate silences script) 
    3. txt file of lyrics to phrases with praatio (check: definition of a phrase, verses instead of choruses decision?)

In [72]:


import os
from os.path import join
from praatio import textgrid
from praatio import praat_scripts
from praatio.utilities.constants import (
    Interval)
import pandas as pd

praat_exe = "/Applications/Praat.app/Contents/MacOS/Praat"
root = "/Users/sarah/Git/eesti_regilaul_corpus/audio"
input_wavs = join(root,"clicks")
input_tg = join(root, "ictus_tier")
output_tg = join(root,"lyric_tier")

SILENCE_LABEL = "x"
SOUND_LABEL = "ictus"

#process the audio 
# from .ogg to wav with sox, then
# for wav in os.listdir(root):
    # if ".wav" not in fn:
    #     continue
    #open audacity, select wav
    # beat finder nyquist 25% sensitivity 
#make initial textgrid: ictus/off


#old method using Logic X Pro MIDI click tracks
def beats_to_textgird(input_wav):

        name = os.path.splitext(fn)[0]
        #remove "beat" so it matches song file
        tgFn = name.strip("beat") + ".TextGrid"
        praat_scripts.annotateSilences(
         praat_exe, join(input_wavs, fn), join(input_tg, tgFn),100,0,-25,0.1,0.1,SILENCE_LABEL,SOUND_LABEL
        )




#TODO add new method for open-source beat marker (txt file to textgrid)
# beat label text file 
# start = b 
# end = (time at index[(b+1)] - (b/2)
# label = ictus or off ictus, every other, starting from first beat (so all start with off-ictus unless the first beat == 0.0)
    

# phrase tier! 
for fn in os.listdir(root):

    # fn.beats_to_textgird()
    # #then, add the phrase tier
    if ".TextGrid" not in fn:
        continue

    tg = textgrid.openTextgrid(fn,True)

    ictus_dict = tg.tierDict
    myTier = textgrid.IntervalTier("phrase",entryList= ictus_dict[1])
    lyric_lines = open(join(root,fn,".txt")).readlines()
    for interval, line in myTier, lyric_lines:
        #TODO first edge case thing
        interval.text = lyric_lines 
    tg.addTier(myTier,[2],"warning")
    tg.save(fn,'long_textgrid')

        #tg.addTier(input_tg,output_tg,"phrase")
#file format: NAME + "beat"

    
def add_lines_intervals(input_tg,input_txt,tier_num):
        tg = textgrid.openTextgrid(input_tg,True)
        myTier = tg.tierDict[tier_num]
        lyrics = open(input_txt,"r")
        lyricList = lyrics.readlines()
        dur = (tg.maxTimestamp/len(lyricList))
        start= 0.0
        length = start + dur
        line = 0
        while length <= tg.maxTimestamp:
            tmpInterval = Interval(start,length,lyricList[line])
            myTier.insertEntry(tmpInterval,'replace','warning')
            start = start+dur
            length = length +dur 
            line = line + 1
        tg.save(join(input, fn),'long_textgrid',True)

#helper function to make modified Textgrids non-destructively
# def addTier(input_tg, output_tg,tier_label):
#         tg = textgrid.openTextgrid(join(input_tg, fn),True)
#         myTier = textgrid.IntervalTier(tier_label, entryList= [('0', tg.maxTimestamp,
#         'test'),],minT=0,maxT=tg.maxTimestamp)
#         tg.addTier(myTier)
#         tg.save(join(output_tg,fn),'long_textgrid', True)



In [139]:
import os
from os.path import join
from praatio import textgrid
from praatio import audio
import pandas as pd
tmptxt = "/Users/sarah/qp_final/060_LabelTrack.txt"
tmpwav = "wavs/060.wav"

inputPath = "/Users/sarah/qp_final/wavs/"
outputPath = join(inputPath, "generated_textgrids/")

if not os.path.exists(outputPath):
    os.mkdir(outputPath)

for fn in os.listdir(inputPath):
    name, ext = os.path.splitext(fn)
    if ext == ".wav":
        wavQObj = audio.WavQueryObj(join(inputPath,fn))
        dur = wavQObj.getDuration()
        #wave = audio.getDuration(tmpwav)
        tg = textgrid.Textgrid(0.0,dur)
        myTier = textgrid.IntervalTier('',[],0.0,dur)
        tg.addTier(myTier)
        tg.save(join(outputPath,(name+".TextGrid")), "long_textgrid", includeBlankSpaces=True)

#later: to put new click tracks in from txt file exported from audacity 
    line_df = pd.read_csv(tmptxt, sep="\t",names=["first","second","B"])
    one_df = line_df["first"]
    one_df.head()
    intervalList = []
    i = 0
    intervalList.append(Interval(0.0,one_df[i],"off"))
    for item in one_df:
        j = i + 1 
        if i % 2 : label = "off"
        else: label = "ictus"
        start = item
        if j < len(one_df): 
            end = one_df[j]
            tmpInterval = Interval(start,end,label)
        intervalList.append(tmpInterval)
        i+=1 
    else: 
        end = dur
        tmpInterval = Interval(start,end,label)



# # Did it work?
# for fn in os.listdir(outputPath):
#     ext = os.path.splitext(fn)[1]
#     if ext != ".TextGrid":
#         continue
#     print(fn)



In [68]:
import pandas as pd
tmptxt = "/Users/sarah/qp_final/060_LabelTrack.txt"
tmpwav = "060.wav"
# def intervalEntry(txt, wav):
#     #make textgrid from wav fn 
#     #add Interval Tier from txt file of floats at item locations
#     line_df = pd.read_csv(join(root,txt))
#     line_df.head()

line_df = pd.read_csv(tmptxt, sep="\t",names=["first","second","B"])
one_df = line_df["first"]
one_df.head()
intervalList = []
i = 0
intervalList.append(0.0,one_df[i],"off")
for item in one_df:
    j = i + 1 
    start = item
    if j < len(one_df): 
        end = one_df[j]
        if i % 2 : label = "off"
        else: label = "ictus"
        tmpInterval = (start,end,label)
        intervalList.append(tmpInterval)
        i+=1 
    else: 
        end = tg.MaxTimeStamp
        tmpInterval = (start,end, )


intervalList



# intervalList

[(0.414, 0.644, 'ictus'),
 (0.644, 1.274, 'off'),
 (1.274, 1.711, 'ictus'),
 (1.711, 2.106, 'off'),
 (2.106, 2.535, 'ictus'),
 (2.535, 2.942, 'off'),
 (2.942, 3.353, 'ictus'),
 (3.353, 3.79, 'off'),
 (3.79, 4.963, 'ictus'),
 (4.963, 5.367, 'off'),
 (5.367, 5.797, 'ictus'),
 (5.797, 5.802, 'off'),
 (5.802, 6.207, 'ictus'),
 (6.207, 6.457, 'off'),
 (6.457, 7.292, 'ictus'),
 (7.292, 8.476, 'off'),
 (8.476, 8.881, 'ictus'),
 (8.881, 9.306, 'off'),
 (9.306, 10.128, 'ictus'),
 (10.128, 10.519, 'off'),
 (10.519, 11.277, 'ictus'),
 (11.277, 11.728, 'off'),
 (11.728, 12.157, 'ictus'),
 (12.157, 12.425, 'off'),
 (12.425, 12.568, 'ictus'),
 (12.568, 13.999, 'off'),
 (13.999, 14.355, 'ictus'),
 (14.355, 14.768, 'off'),
 (14.768, 15.519, 'ictus'),
 (15.519, 15.721, 'off'),
 (15.721, 15.963, 'ictus'),
 (15.963, 16.798, 'off'),
 (16.798, 17.197, 'ictus'),
 (17.197, 17.587, 'off'),
 (17.587, 17.988, 'ictus'),
 (17.988, 18.766, 'off'),
 (18.766, 19.417, 'ictus'),
 (19.417, 19.568, 'off'),
 (19.568, 20.

4. forced aligner: word level (check: definition of word)
5. syllable level (coded for stress and quanitity!) with varbomorf manual check: definition of a syllable (use ictus/off-ictus, duh!)

In [158]:
from praatio import textgrid
import string
import pandas as pd
grids_in = "/Users/sarah/qp_final/wavs/generated_textgrids/"

# def add_syll_tier(grids,out):
#     if not os.path.exists(out): os.mkdir(out)
#     for fn in os.listdir(grids):
#sylly_name = (fn.strip(".TextGrid") + "_sillies.csv")
sylly_name = ("/Users/sarah/qp_final/wavs/generated_textgrids/055_sillies.csv")

# if ".TextGrid" not in fn: 
#     continue
tg = textgrid.openTextgrid(join(grids_in, "055.TextGrid"), True)
wordTier = tg.tierDict[""]
tmpTier = wordTier.new("varbo")

syllies = pd.read_csv(sylly_name)
print(syllies)
    

    
    



#dict_in = "/Users/sarah/Git/eesti_regilaul_corpus/grids/syll/009_sillies.csv"
grid_out = "wavs/generated_textgrids/sylls/"
#add_syll_tier(grids_in,grid_out)

   

          Laula  \
0         laula   
1      suukkene   
2         liigu   
3         linnu   
4    keelekkene   
5         mõlgu   
6         marja   
7    meelekkene   
8       iluttse   
9   südämekkene   
10         Küll   
11           sa   
12         siis   
13         saad   
14        vaitt   
15         olla   
16          kui   
17         alla   
18       mustta   
19        mulla   
20        valge   
21       lauade   
22       vahele   
23         kena   
24      kirsttu   
25       keskke   
26         elle   

   [[{'syllable': 'lau', 'quantity': 2, 'accent': 1}, {'syllable': 'la', 'quantity': 1, 'accent': 0}]]  
0   [[{'syllable': 'lau', 'quantity': 2, 'accent':...                                                   
1   [[{'syllable': 'suuk', 'quantity': 3, 'accent'...                                                   
2   [[{'syllable': 'lii', 'quantity': 2, 'accent':...                                                   
3   [[{'syllable': 'lin', 'quantity': 2, 'accen

# segmental level (run from syllable level) (manual check: definition of each segment and the segment's environmental criteria to be included in analysis)

## The measurement process is described:
1. temporal aspects: 
    1. vowel duration
    2. consonant duration 
    3. syllable duration 
2. spectral aspects: 
    1. vowel space
    2. spectral tilt

# Results
## Ictus and Off-Ictus (song prominence)
### temporal 
### spectral 
## Stressed and unstressed syllables (word-level prominence)
### temporal 
### spectral 
## Conflicts: stressed syllables in off-ictus position
### temporal
### spectral 
## Conflicts: unstressed syllables in off-ictus position
### temporal
### spectral
# Discussion 
# Conclusion 
